add a new field to the index using a different analyzer

In [1]:
curl -XPUT 'localhost:9200/my_index/_mapping/my_type?pretty' -H 'Content-Type: application/json' -d'
{
    "my_type": {
        "properties": {
            "english_title": {
                "type":     "string",
                "analyzer": "english"
            }
        }
    }
}
'

{
  "acknowledged" : true
}


analyze using the standard analyzer

In [2]:
curl -XGET 'localhost:9200/my_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "field": "title",   
  "text": "Foxes"
}
'

{
  "tokens" : [
    {
      "token" : "foxes",
      "start_offset" : 0,
      "end_offset" : 5,
      "type" : "<ALPHANUM>",
      "position" : 0
    }
  ]
}


analyze using the english analyzer

In [3]:
curl -XGET 'localhost:9200/my_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "field": "english_title",   
  "text": "Foxes"
}
'

{
  "tokens" : [
    {
      "token" : "fox",
      "start_offset" : 0,
      "end_offset" : 5,
      "type" : "<ALPHANUM>",
      "position" : 0
    }
  ]
}


see how the match query determines which analyzer to use, using the validate-query API

In [4]:
curl -XGET 'localhost:9200/my_index/my_type/_validate/query?explain&pretty' -H 'Content-Type: application/json' -d'
{
    "query": {
        "bool": {
            "should": [
                { "match": { "title":         "Foxes"}},
                { "match": { "english_title": "Foxes"}}
            ]
        }
    }
}
'

{
  "valid" : true,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "explanations" : [
    {
      "index" : "my_index",
      "valid" : true,
      "explanation" : "+(title:foxes english_title:fox) #(#_type:my_type)"
    }
  ]
}
